In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

データファイルの準備

In [ ]:
# import requests
# import zipfile
# import io

# # ZIPファイルのURL
# url = "http://chasen.org/~daiti-m/gpbook/data/gpr-data.zip"

# # ダウンロードしてメモリ上で解凍
# response = requests.get(url)
# if response.status_code == 200:
#     with zipfile.ZipFile(io.BytesIO(response.content)) as z:
#         z.extractall("./../data/chap1")

回帰の実装

In [ ]:
def linear_regression(X, y):
    """
    線形回帰の実装
    X: 入力データ (n_samples, n_features)
    y: 出力データ (n_samples,)
    """
    y = y.reshape(-1, 1)  # yを列ベクトルに変形
    X = np.vstack([np.ones(X.shape[0]),X.T]).T # バイアス項を追加
    b = X.T @ y
    w = np.linalg.inv(X.T @ X) @ b  # 重みの計算
    return w

単回帰

In [ ]:
def plot_simple(X, y, w, figname):
    """
    単回帰の結果をプロット
    X: 入力データ (n_samples,)
    y: 出力データ (n_samples,)
    w: 重み (2,)
    """
    xmin,xmax = -5,5
    ymin,ymax = -5,5
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    ax.plot([xmin,xmax],[0,0],'k',linewidth=1, zorder=0)
    ax.plot([0,0],[ymin,ymax],'k',linewidth=1, zorder=0)
    ax.scatter(X, y, label='Data', color="black", zorder=1)
    fig.savefig(f"./../figures/chap1/{figname}_data.png")

    xx = np.linspace(xmin, xmax, 10)
    yy = w[0] + w[1] * xx
    ax.plot(xx, yy, label=f'$y=a+bx$', zorder=2)
    # ax.plot(xx, yy, label=f'$y=a+bx$ \n $(a={w[0][0]:.2f}, b={w[1][0]:.2f})$', zorder=2)

    # ax.legend()
    fig.savefig(f"./../figures/chap1/{figname}.png")
    plt.show()

In [ ]:
# データの読み込み
data = np.loadtxt("./../data/chap1/data/simple1.dat")
y1 = data[:, 0]
X1 = data[:, 1]

# 単回帰の実行
w1 = linear_regression(X1, y1)
print(w1)

# 結果のプロット
plot_simple(X1, y1, w1, "simple1")

In [ ]:
# データの読み込み
data = np.loadtxt("./../data/chap1/data/simple2.dat")
y1_2 = data[:, 0]
X1_2 = data[:, 1]

# 単回帰の実行
w1_2 = linear_regression(X1_2, y1_2)
print(w1)

# 結果のプロット
plot_simple(X1_2, y1_2, w1_2, "simple2")

重回帰

In [ ]:
def plot_data(X, y):
    """
    重回帰の結果をプロット
    X: 入力データ (n_samples, n_features)
    y: 出力データ (n_samples,)
    w: 重み (n_features + 1,)
    """
    scatter = go.Scatter3d(
        x=X[:, 0],
        y=X[:, 1],
        z=y,
        mode='markers',
        marker=dict(size=5, color='black'),
        name='Data'
    )
    layout = go.Layout(
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='y'
        ),
    )
    fig = go.Figure(data=[scatter], layout=layout)
    fig.show()

In [ ]:
def plot_multiple(X, y, w):
    """
    重回帰の結果をプロット
    X: 入力データ (n_samples, n_features)
    y: 出力データ (n_samples,)
    w: 重み (n_features + 1,)
    """
    scatter = go.Scatter3d(
        x=X[:, 0],
        y=X[:, 1],
        z=y,
        mode='markers',
        marker=dict(size=5, color='black'),
        name='Data'
    )
    xx, yy = np.meshgrid(np.linspace(X[:, 0].min(), X[:, 0].max(), 10),
                         np.linspace(X[:, 1].min(), X[:, 1].max(), 10))
    zz = w[0] + w[1] * xx + w[2] * yy
    surface = go.Surface(
        x=xx,
        y=yy,
        z=zz,
        opacity=0.5,
        colorscale=[[0, 'blue'], [1, 'blue']],
        name='Regression Plane',
        showscale=False
    )
    layout = go.Layout(
        scene=dict(
            xaxis_title='x1',
            yaxis_title='x2',
            zaxis_title='y'
        ),
    )
    fig = go.Figure(data=[scatter, surface], layout=layout)
    # fig.write_image("./../figures/chap1/multiple.png")
    fig.show()

In [ ]:
# データの読み込み
data = np.loadtxt("./../data/chap1/multiple.dat")
X2 = data[:, 0:2]
y2 = data[:, 2]
plot_data(X2, y2)

# 重回帰の実行
w2 = linear_regression(X2, y2)
print(w2)

# 結果のプロット
plot_multiple(X2, y2, w2)

非線形基底関数

In [ ]:
def phi (x):
    return np.array([x, x*x, np.sin(x), np.cos(x)])

def lm (xx,w):
    return [w.T @ np.array([1, *phi(x)]) for x in xx]

In [ ]:
def plot_nonlinear(X, y, w):
    """
    線形回帰の結果をプロット
    X: 入力データ (n_samples,)
    y: 出力データ (n_samples,)
    w: 重み (2,)
    """
    xmin,xmax = -4,4
    ymin,ymax = -2,2
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.scatter(X, y, label='Data', color="black")

    xx = np.linspace(xmin, xmax, 50)
    yy = lm(xx, w)
    ax.plot(xx, yy)

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    # ax.legend()
    fig.savefig("./../figures/chap1/nonlinear.png")
    plt.show()

In [ ]:
# データの読み込み
data = np.loadtxt("./../data/chap1/nonlinear.dat")
y3 = data[:, 1]
X3 = data[:, 0]
X3_trans = np.array([phi(x) for x in X3.T])

# 回帰の実行
w3 = linear_regression(X3_trans, y3)
print(w3)

# 結果のプロット
plot_nonlinear(X3, y3, w3)

In [ ]:
def plot_nonlinear_2(X, y, w):
    xmin,xmax = -4,4
    ymin,ymax = -2,2
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.scatter(X, y, label='Data', color="black")

    xx = np.linspace(xmin, xmax, 50)
    yy = w[0] + w[1] * xx
    ax.plot(xx, yy)

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_aspect('equal')
    # ax.legend()
    fig.savefig("./../figures/chap1/nonlinear_2.png")
    plt.show()


# 回帰の実行
w3_2 = linear_regression(X3, y3)
print(w3)

# 結果のプロット
plot_nonlinear_2(X3, y3, w3_2)

Ridge回帰

In [ ]:
def linear_regression_ridge(X, y, alpha=0.1):
    """
    線形回帰の実装
    X: 入力データ (n_samples, n_features)
    y: 出力データ (n_samples,)
    alpha: Ridge回帰の係数
    """

    y = y.reshape(-1, 1)  # yを列ベクトルに変形
    X = np.vstack([np.ones(X.shape[0]),X.T]).T # バイアス項を追加
    b = X.T @ y
    w = np.linalg.inv(X.T @ X + alpha*np.diag(np.ones(X.shape[1]))) @ b  # 重みの計算
    return w

In [ ]:
def plot_simple_ridge(X, y, w_simple, w_ridge_list, alpha_list):
    """
    単回帰の結果をプロット
    X: 入力データ (n_samples,)
    y: 出力データ (n_samples,)
    w: 重み (2,)
    """
    xmin,xmax = -5,5
    ymin,ymax = -5,5
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot([xmin,xmax],[0,0],'k',linewidth=1)
    ax.plot([0,0],[ymin,ymax],'k',linewidth=1)
    ax.scatter(X, y, label='Data')

    xx = np.linspace(xmin, xmax, 10)
    yy_simple = w_simple[0] + w_simple[1] * xx
    ax.plot(xx, yy_simple, label='simple')
    
    for i in range(len(w_ridge_list)):
        yy_ridge = w_ridge_list[i][0] + w_ridge_list[i][1] * xx
        ax.plot(xx, yy_ridge, label=f'ridge ($\\alpha = {alpha_list[i]}$)')

    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend()
    plt.show()

In [ ]:
# データの読み込み
data = np.loadtxt("./../data/chap1/data/simple1.dat")
y1 = data[:, 0]
X1 = data[:, 1]

# Ridge回帰の実行
alpha_list = [0.1, 1, 10, 100]
w4_list = [linear_regression_ridge(X1, y1, alpha) for alpha in alpha_list]
print(w4_list)

# # 結果のプロット
plot_simple_ridge(X1, y1, w1, w4_list, alpha_list)

In [ ]:
X5 = np.array([[2, 4], [3, 6.1], [4, 7.9]])
y5 = np.array([3, -1, 3])

# 重回帰の実行
w5 = linear_regression(X5, y5)

# 結果のプロット
plot_multiple(X5, y5, w5)

In [ ]:
# Ridge回帰の実行
w6 = linear_regression_ridge(X5, y5)

# 結果のプロット
plot_multiple(X5, y5, w6)

Lasso回帰

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# Lasso回帰の実行
clf = Lasso(alpha=0.1)
clf.fit(X5, y5)
w7 = [clf.intercept_, *clf.coef_]

# 結果のプロット
plot_multiple(X5, y5, w7)

通常の重回帰、Ridge回帰、Lasso回帰の比較
(参考：https://boritaso-blog.com/lasso_and_ridge/)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import pandas as pd

In [ ]:
def phi2 (x):
    return np.array([x**i for i in range(1, 26)])

def lm2 (xx,w):
    return [w.T @ np.array([1, *phi2(x)]) for x in xx]

# データ生成
np.random.seed(0)
X_train = np.linspace(-3, 3, 50)  # 10個のトレーニングデータ点を生成
y_train = np.sin(X_train) + np.random.normal(0, 0.2, size=X_train.shape)  # 正弦関数にノイズを追加

# 真の関数
xx = np.linspace(-3, 3, 100)
yy_true = np.sin(xx)

xmin,xmax = -3,3
ymin,ymax = -2,2
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot([xmin,xmax],[0,0],'k',linewidth=1)
ax.plot([0,0],[ymin,ymax],'k',linewidth=1)
ax.scatter(X_train, y_train, label='Data', color='black')
ax.plot(xx, yy_true, label='true')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend()
fig.savefig("./../figures/chap1/compare_data.png")
plt.show()


# 多項式特徴量の追加
poly_degree = 25  # 高次の多項式を使用して過学習を起こす
poly_features = PolynomialFeatures(degree=poly_degree, include_bias=False)

X_train_poly = poly_features.fit_transform(X_train[:, np.newaxis])

# モデルの学習
w_lr = linear_regression(X_train_poly, y_train) # 通常の重回帰
w_ridge = linear_regression_ridge(X_train_poly, y_train, alpha=0.1) # Ridge回帰
clf = Lasso(alpha=0.01)
clf.fit(X_train_poly, y_train)
w_lasso = np.array([clf.intercept_, *clf.coef_]) # Lasso回帰

# 回帰係数の取得
df_w_lr = pd.DataFrame({"Normal Coefficients": w_lr.reshape(-1)})
df_w_ridge = pd.DataFrame({"Ridge Coefficients": w_ridge.reshape(-1)})
df_w_lasso = pd.DataFrame({"Lasso Coefficients": w_lasso})
# 係数データフレームの結合
df_coef = pd.concat([df_w_lr, df_w_ridge, df_w_lasso], axis=1)

# 予測
yy_lr = lm2(xx, w_lr)
yy_ridge = lm2(xx, w_ridge)
yy_lasso = lm2(xx, w_lasso)

# MSEの計算
mse_normal = mean_squared_error(yy_true, yy_lr)
mse_ridge = mean_squared_error(yy_true, yy_ridge)
mse_lasso = mean_squared_error(yy_true, yy_lasso)

# 結果の描画
xmin,xmax = -3,3
ymin,ymax = -2,2
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot([xmin,xmax],[0,0],'k',linewidth=1)
ax.plot([0,0],[ymin,ymax],'k',linewidth=1)
ax.scatter(X_train, y_train, label='Data', color='black')
ax.plot(xx, yy_true, label='true')
ax.plot(xx, yy_lr, label='multiple', linestyle='--')
ax.plot(xx, yy_ridge, label='ridge', linestyle='--')
ax.plot(xx, yy_lasso, label='lasso', linestyle='--')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.legend()
fig.savefig("./../figures/chap1/compare.png")
plt.show()

# 回帰係数の出力
print("Regression Coefficients:\n", df_coef)

# MSEの出力
print("\nNormal Model MSE:", mse_normal)
print("Ridge Model MSE:", mse_ridge)
print("Lasso Model MSE:", mse_lasso)